In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

In [2]:
# load CSV's
df_train = pd.read_csv('/home/malmason/datasets/siim-isic-melanoma-classification/cleaned_csv/train_pp.csv')
df_test = pd.read_csv('/home/malmason/datasets/siim-isic-melanoma-classification/cleaned_csv/test_pp.csv')

In [3]:
# Point to image folders
img_train_folder = '/home/malmason/datasets/siim-isic-melanoma-classification/ycbcr/norm/train/'
img_test_folder = '/home/malmason/datasets/siim-isic-melanoma-classification/ycbcr/norm/test/'

In [4]:
df_train_csv = df_train

In [5]:
df_train_csv = df_train_csv.append(df_train_csv.loc[df_train_csv['target'] == 1])
df_train_csv = df_train_csv.append(df_train_csv.loc[df_train_csv['target'] == 1])

In [6]:
df_train_csv = df_train_csv.sample(frac=1).reset_index(drop=True)

In [7]:
df_train_csv.shape

(34878, 17)

In [8]:
# Used for looping through image columns
X_train_img = df_train_csv['image_name']

In [9]:
X_train_csv = df_train_csv.drop(['target', 'image_name'], axis=1).values
y_train_csv = df_train_csv['target'].values

In [10]:
print(X_train_csv.shape, y_train_csv.shape)

(34878, 15) (34878,)


In [11]:
sizing = 224 - 2            # conv1
sizing = (sizing - 2) / 2   # conv2  
sizing = (sizing - 2)       # conv3
sizing = (sizing - 2) / 2   # conv4
sizing = (sizing - 2)       # conv5
sizing = (sizing - 2) / 2   # conv6
sizing = (sizing - 2)       # conv7
sizing = (sizing - 2) / 2   # conv8
sizing = (sizing - 2) /2    # conv9
print(512*(int(sizing)*int(sizing)))

8192


In [12]:
sizing = 224 - 4            # conv1
sizing = (sizing - 4) / 2   # conv2  
sizing = (sizing - 4) / 2   # conv4
sizing = (sizing - 4) / 2   # conv6
sizing = (sizing - 4) / 2   # conv8
sizing = (sizing - 4) / 2   # conv9
print(512*(int(sizing)*int(sizing)))

4608


In [13]:
class CNN(nn.Module):
  def __init__(self, n_output_neurons):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(3 , 32,   kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(32, 32,   kernel_size=3, stride=1)
    self.bn1   = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32 , 64,  kernel_size=3, stride=1)
    self.conv4 = nn.Conv2d(64 , 64,  kernel_size=3, stride=1)

    self.conv5 = nn.Conv2d(64 , 128, kernel_size=3, stride=1)
    self.conv6 = nn.Conv2d(128 ,128, kernel_size=3, stride=1)

    self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=1)
    self.conv8 = nn.Conv2d(256 ,256, kernel_size=3, stride=1)
    self.bn4   = nn.BatchNorm2d(256)
    self.conv9 = nn.Conv2d(256, 512, kernel_size=3, stride=1)


    self.conv11 = nn.Conv2d(3 , 32,   kernel_size=5, stride=1)
    self.bn11   = nn.BatchNorm2d(32)
    self.conv21 = nn.Conv2d(32 , 64,  kernel_size=5, stride=1)

    self.conv31 = nn.Conv2d(64 , 128, kernel_size=5, stride=1)

    self.conv41 = nn.Conv2d(128, 256, kernel_size=5, stride=1)

    self.conv51 = nn.Conv2d(256, 512, kernel_size=5, stride=1)
    self.bn51   = nn.BatchNorm2d(512)
    
    self.data1 = nn.Linear(15, 128)
    self.data2 = nn.Linear(128, 16)
    
    self.img_fc1   = nn.Linear(8192+4608,128)
    self.img_fc2   = nn.Linear(128,32)
    self.img_fc3   = nn.Linear(32,16)
    
    self.fc1_conc  = nn.Linear(16,n_output_neurons)

  def forward(self, x, xd):
    
    xa = F.relu(self.conv1(x),2)
    xa = F.relu(F.max_pool2d(self.conv2(xa),2))
    xa = self.bn1(xa)
    xa = F.relu(self.conv3(xa),2)
    xa = F.relu(F.max_pool2d(self.conv4(xa),2))

    xa = F.relu(self.conv5(xa),2)
    xa = F.relu(F.max_pool2d(self.conv6(xa),2))

    xa = F.relu(self.conv7(xa),2)
    xa = F.relu(F.max_pool2d(self.conv8(xa),2))
    xa = self.bn4(xa)
    xa = F.relu(F.max_pool2d(self.conv9(xa),2))


    xb = F.relu(F.max_pool2d(self.conv11(x),2))
    xb = self.bn11(xb)
    xb = F.relu(F.max_pool2d(self.conv21(xb),2))

    xb = F.relu(F.max_pool2d(self.conv31(xb),2))

    xb = F.relu(F.max_pool2d(self.conv41(xb),2))

    xb = F.relu(F.max_pool2d(self.conv51(xb),2))
    xb = self.bn51(xb)
    
    xa = xa.view(xa.size(0), -1)
    xb = xb.view(xb.size(0), -1)
    
    xi = torch.cat((xa, xb), dim=1)
    
    xi = F.leaky_relu(self.img_fc1(xi))
    xi = F.dropout(xi, p=0.3)
    
    xi = F.leaky_relu(self.img_fc2(xi))
    xi = F.dropout(xi, p=0.2)
    xi = F.leaky_relu(self.img_fc3(xi))
    
    xd = F.leaky_relu(self.data1(xd))
    xd = F.dropout(xd, p=0.2)
    xd = F.leaky_relu(self.data2(xd))
    
    x = torch.cat((xi, xd), dim=1)
    
    x = self.fc1_conc(xd)

    return x

In [14]:
n_output_neurons = 1

In [15]:
model = CNN(n_output_neurons)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv11): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (bn11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv21): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv31): Conv2d(64, 128, kernel_size=(5, 5)

In [17]:
X_train_image = []
for image_get in X_train_img:
    img_train = cv2.imread(img_train_folder + '{}.jpg'.format(image_get))
    '''
    rand_num = np.random.uniform(low=1, high=1.2)
    M = cv2.resize(img_train, None, fx= rand_num, fy= rand_num, interpolation= cv2.INTER_LINEAR)
    
    H_crop = ((224*rand_num)-224)/2
    V_crop = ((224*rand_num)-224)/2
    
    C = M[np.int(H_crop):np.int(M.shape[0]-H_crop),np.int(V_crop):np.int(M.shape[1]-np.int(V_crop))]
    img_train= cv2.resize(C,(224,224))
    '''
    X_train_image.append(img_train)

In [18]:
X_train_image = np.array(X_train_image)
X_train_image = X_train_image.astype(np.float32)

In [19]:
Y_train = np.array(y_train_csv)

In [20]:
X_train_image = X_train_image / 255

In [21]:
print(f'Train shape: {X_train_image.shape}')

Train shape: (34878, 224, 224, 3)


In [22]:
X_train_image_t = np.transpose(X_train_image, (0,3,1,2))

In [23]:
input_train = torch.from_numpy(X_train_image_t)

In [24]:
target_train = torch.from_numpy(Y_train).reshape(-1,1).float()

In [25]:
X_train_csv = X_train_csv.astype(np.float32)

In [26]:
input_train_data = torch.from_numpy(X_train_csv)

In [27]:
n_epochs = 2
train_losses = []
val_losses = []

In [28]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

In [29]:
batch_size = 128
training_set = torch.utils.data.TensorDataset(input_train, input_train_data, target_train)
train_loader = torch.utils.data.DataLoader(dataset=training_set, 
                                           batch_size=batch_size,
                                           num_workers=2,
                                           shuffle=True)

In [30]:
def binary_acc(output_pred, target):
    output_pred_tag = torch.round(torch.sigmoid(output_pred))

    correct_results_sum = (output_pred_tag == target).sum().float()
    acc = correct_results_sum/target.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [31]:
def batch_gd(model, criterion, optimizer, train_loader, n_epochs):
    train_losses = np.zeros(n_epochs)
    train_accuracy = np.zeros(n_epochs)
    
    for it in range(n_epochs):
        t0 = datetime.now()
        
        train_loss = []
        train_acc = []
        
        for inputs, inputs_data, targets in tqdm(train_loader):
            inputs, inputs_data, targets = inputs.to(device), inputs_data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs, inputs_data)

            loss = criterion(outputs, targets)
            acc = binary_acc(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_acc.append(acc.item())
            
        train_loss = np.mean(train_loss)
        train_acc = np.mean(train_acc)
        
        train_losses[it] = train_loss
        train_accuracy[it] = train_acc

        dt = datetime.now() -t0

        print(f'Epoch {it+1}/{n_epochs}, Time: {dt}, Train Loss: {train_loss:.4f}, Train_acc: {train_acc}')
    
    return train_losses, train_accuracy

In [32]:
train_losses, train_accuracy  = batch_gd(
    model, criterion, optimizer, train_loader, n_epochs)

  0%|          | 0/273 [00:00<?, ?it/s]

Epoch 1/2, Time: 0:00:41.983246, Train Loss: 0.3789, Train_acc: 93.10989010989012


100%|██████████| 273/273 [00:42<00:00,  6.48it/s]

Epoch 2/2, Time: 0:00:42.160823, Train Loss: 0.2251, Train_acc: 93.31868131868131


## Start of test data

In [33]:
df_test_csv = df_test

In [34]:
X_test_img = df_test_csv['image_name']

In [35]:
X_test_csv = df_test_csv.drop(['image_name'], axis=1).values

In [36]:
X_test_image = []
for image_get in X_test_img:
    img_test = cv2.imread(img_test_folder + '{}.jpg'.format(image_get))

    X_test_image.append(img_test)

In [37]:
X_test_image = np.array(X_test_image)
X_test_image = X_test_image.astype(np.float32)

In [38]:
X_test_image = X_test_image / 255

In [39]:
X_test_image_t = np.transpose(X_test_image, (0,3,1,2))

In [40]:
len(X_test_image_t)

10982

In [41]:
input_test = torch.from_numpy(X_test_image_t)

In [42]:
X_test_csv = X_test_csv.astype(np.float32)

In [43]:
input_test_data = torch.from_numpy(X_test_csv)

In [45]:
batch_size = 128
test_set = torch.utils.data.TensorDataset(input_test, input_test_data)
test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                           batch_size=batch_size,
                                           num_workers=0,
                                           shuffle=False)

In [48]:
predicted_outputs = []
for inputs, inputs_data in tqdm(test_loader):
    inputs, inputs_data = inputs.to(device), inputs_data.to(device)
            
    optimizer.zero_grad()
    outputs = model(inputs, inputs_data)
    outputs_sigmoid = torch.sigmoid(outputs)
    outputs_sigmoid_numpy = outputs_sigmoid.detach().cpu().numpy()
    predicted_outputs.append(outputs_sigmoid_numpy)

100%|██████████| 86/86 [00:13<00:00,  6.61it/s]


In [51]:
predicted_outputs = np.array(predicted_outputs)

In [68]:
predicted_outputs_total = []
for count in range(len(predicted_outputs)):
    for val in predicted_outputs[count]:
        predicted_outputs_total.append(val)
predicted_outputs_total = np.array(predicted_outputs_total)

In [76]:
print(predicted_outputs_total[0])

[5.023782e-06]


In [ ]:
from sklearn.metrics import classification_report
import itertools

In [ ]:
print(classification_report(train_predictions_np_out, train_targets_np_out))

In [ ]:
print(classification_report(val_predictions_np_out, val_targets_np_out))

In [ ]:
max(val_targets_np_out)